In [1]:
import os
import re
import pdfplumber
import numpy as np
import pandas as pd 
import json

import plotly.graph_objs as go
import datetime as dt
import plotly as py
from plotly.subplots import make_subplots

In [2]:
# Enable multiple output per cell. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.chdir(r'C:\Users\Owner\OneDrive\Digital Mosaic\Product\Solutions\Vibrant')

### Extract Pages
Goal is create table with sample sections for <font color=cyan>Collateral Quality Test </font>. 
1. Extract relevant pages from pdf then combine into one string.
2. Search <font color=cyan> "Collateral Quality Test" & "Collections Account" </font>, with the latter being the section that follows Collateral Quality. 
3. Use regex to isolate bullet points and store values in table. 

In [5]:
start_page = 22
end_page = 35 
text_list = [] 
with pdfplumber.open('OHA XII Indenture.pdf') as pdf:
    for i in np.arange(start_page,end_page):
        page = pdf.pages[i]
        text = page.extract_text()
        text_list.append(text)

In [8]:
# combine document string 
doc_str = text_list[0]
for i in np.arange(1, len(text_list)):
    doc_str = doc_str + text_list[i]
len(doc_str) 

36980

In [4]:
doc_str = text_list[0] + text_list[1]
end_doc = len(doc_str) 

1907

In [6]:
# Calc lines on one page 
pg1 = text_list[0]
num_lines = 0 
for i, line in enumerate(pg1.split('\n')):
    num_lines += 1
num_lines

34

In [7]:
# Test isolateing on search string and end string 
for i, line in enumerate(doc_str.split('\n')):
    col = line.find("Collateral Quality Test")
    if col == 1:
        match_line = i
        print(i, line)
match_line 

15 “Collateral Quality Test”:  A test satisfied if, as of any Measurement Date, for so 


15

In [46]:
# Alternative for finding "Collaterla Quality Test"
search_string = '“Concentration Limitations”'
    # collat_st_re = re.compile(r'“Collateral Quality Test”')
collat_st_re = re.compile(search_string)
# "(.*?)"
collat_match = collat_st_re.finditer(doc_str)
for match in collat_match:
    start_collat = match.span()[0]
    end_collat = match.span()[1]
    start_collat
    doc_str[start_collat:end_collat]

2647

'“Concentration Limitations”'

### Exploring end of section 
1. Explicitly typing name of next section. 2
2. search on quotation (more generalized) 

In [9]:
# Alternative for finding "Collectin Account"
collect_st_re = re.compile(r'“Collection Account”')
# collat_st_re2 = re.compile(search_string)
# "(.*?)"
collect_match = collect_st_re.finditer(doc_str)
for match in collect_match:
    start_collect = match.span()[0]
    end_collect = match.span()[1]
    start_collect
    doc_str[start_collect:end_collect]

1705

'“Collection Account”'

In [48]:
# find next open quotation: “ 
first_word_len = len(search_string)
doc_sub = doc_str[(start_collat+1):end_doc]   #substring of section to end of doc
next_loc = doc_sub.find('“')
doc_sub[next_loc:next_loc+11]
start_next = start_collat + next_loc + 1 
start_next

''

2647

Create substring with just Collateral Quality Section 

In [38]:
sub_doc_str = doc_str[start_collat:start_next]
sub_doc_str2 = sub_doc_str.replace('\n','') # same string with newlines removed

In [39]:
# new_re = re.compile(r"\([vix]+\).*;")
new_re = re.compile(r"\([vix]+\)")
matches = new_re.finditer(sub_doc_str)
bullet_ranges = []
for match in matches:
    print(match.span()) 
    bullet_ranges.append(match.span())
    start_roman = match.span()[0]
    end_roman = match.span()[1]
    sub_doc_str[start_roman:end_roman]

(400, 403)


'(i)'

(437, 441)


'(ii)'

(478, 483)


...

'(vi)'

(663, 668)


'(vii)'

(763, 769)


'(viii)'

Append all conditions except last for Collateral Quality through loop, then append the last condition (ends at start of "Collection Account")

In [40]:
collat_qual_def = [] 
for i in np.arange(0, (len(bullet_ranges)-1)):
    start_loc = bullet_ranges[i][0]
    end_loc = bullet_ranges[i+1][0] - 1 
    collat_qual_def.append(sub_doc_str[start_loc:end_loc])

In [41]:
lstart_loc = bullet_ranges[len(bullet_ranges)-1][0]
collat_qual_def.append(sub_doc_str[lstart_loc:start_collect])

In [42]:
# remove any semi colons and new line
collat_qual_def2 = []
for item in collat_qual_def:
    str1 = item.replace('\n', '')
    str1 = str1.replace(';','')
    collat_qual_def2.append(str1)
collat_qual_def2

['(i)  the Minimum Fixed Coupon Test ',
 '(ii)  the Minimum Floating Spread Test ',
 '(iii)  the Maximum Moody’s Rating Factor Test ',
 '(iv)  the Moody’s Diversity Test ',
 '(v)  the Minimum Weighted Average Moody’s Recovery Rate Test ',
 '(vi)  the Weighted Average Life Test ',
 '(vii)  the S&P Minimum Weighted Average Recovery Rate Test so long as S&P is a Rating Agency and ',
 '(viii)  during the Reinvestment Period only, the S&P CDO Monitor Test so long as S&P is a Rating Agency. ']

In [16]:
conditions_dict = {}
conditions_dict['Collateral Quality Test'] = collat_qual_def2

Export to Excel

In [25]:
df = pd.DataFrame(data=conditions_dict)
df.to_excel('df_test.xlsx')